In [ ]:
#sources
#https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/pinecone.html#
#https://www.youtube.com/watch?v=aywZrzNaKjs - did not work well used only bootstraping code 

In [1]:
!pip install pinecone-client python-dotenv langchain

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.document_loaders import TextLoader

In [4]:
from langchain.document_loaders import TextLoader
loader = TextLoader('ira-bill.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

Created a chunk of size 1049, which is longer than the specified 1000
Created a chunk of size 4483, which is longer than the specified 1000
Created a chunk of size 4116, which is longer than the specified 1000
Created a chunk of size 3833, which is longer than the specified 1000
Created a chunk of size 2301, which is longer than the specified 1000
Created a chunk of size 3999, which is longer than the specified 1000
Created a chunk of size 3809, which is longer than the specified 1000
Created a chunk of size 3920, which is longer than the specified 1000
Created a chunk of size 3546, which is longer than the specified 1000
Created a chunk of size 1129, which is longer than the specified 1000
Created a chunk of size 1961, which is longer than the specified 1000
Created a chunk of size 1153, which is longer than the specified 1000
Created a chunk of size 2317, which is longer than the specified 1000
Created a chunk of size 2862, which is longer than the specified 1000
Created a chunk of s

In [6]:
import pinecone 
import os 

# initialize pinecone
pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),
    environment=os.getenv('PINECONE_ENV')
)

pinecone.whoami()

WhoAmIResponse(username='33fceb3', user_label='default', projectname='8c35f7c')

In [18]:
import pinecone 
import os 

# initialize pinecone
pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),
    environment=os.getenv('PINECONE_ENV')
)

index_name = "langchain-demo"

if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=1536, metric="dotproduct", pods=1) 

docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)

# if you already have an index, you can load it like this
# docsearch = Pinecone.from_existing_index(index_name, embeddings)

query = "how much money is granted to airports in the bill?"
docs = docsearch.similarity_search(query)

In [19]:
print(docs[0].page_content)

(c) Cost Share.--The Federal share of the cost of a project carried 
out using grant funds under subsection (a) shall be 75 percent of the 
total proposed cost of the project, except that such Federal share shall 
increase to 90 percent of the total proposed cost of the project if the 
eligible entity is a small hub airport or nonhub airport, as such terms 
are defined in section 47102 of title 49, United States Code.
    (d) <<NOTE: Deadline.>>  Fuel Emissions Reduction Test.--For 
purposes of clause (ii) of subsection (e)(7)(E), the Secretary shall, 
not later than 2 years after the date of enactment of this section, 
adopt at least 1 methodology for testing lifecycle greenhouse gas 
emissions that meets the requirements of such clause.


In [ ]:
#apply above logic to below structure and UI 
# dashboard > bind to ui controls > bind to collect messages > call get_completion_from_messages > 

In [15]:
import openai
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [16]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [17]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""You are a data analyst, you help the user get insights from the document"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)